In [18]:
# for changing directory
import os

# for opening and working with the .grib/.nv files
import cfgrib
import cftime
import warnings
import xarray as xr
xr.set_options(display_style='html')
warnings.simplefilter("ignore")

# for calculations
import numpy as np
from sklearn.metrics import mean_squared_error
import xesmf as xe                             # for regridding
from scipy.interpolate import interp1d

from scipy.optimize import differential_evolution

from scipy import integrate

from scipy import stats                        # for regression statistics
from scipy.stats import linregress             # for linear regression
from scipy.stats import pearsonr               

# for nice displaying
import pandas as pd               
from IPython.display import HTML

# for nice plots
import intake
%matplotlib inline
from glob import glob
import matplotlib.pyplot as plt
import matplotlib.cm as cm                      # for the color map
import matplotlib.colors as mcolors

# for skiping running a certain cell
from IPython.core.magic import register_cell_magic

@register_cell_magic
def comment(line, cell):
    return

import dask

In [2]:
cat_url = "https://storage.googleapis.com/cmip6/pangeo-cmip6.json"
col = intake.open_esm_datastore(cat_url)
col

,unique
activity_id,18
institution_id,36
source_id,88
experiment_id,170
member_id,657
table_id,37
variable_id,700
grid_label,10
zstore,514818
dcpp_init_year,60


In [3]:
# Selecting the parameters I need
# Same filters as in the CMIP6 page (https://esgf-data.dkrz.de/search/cmip6-dkrz/)

experiment_id = ['piControl', 'abrupt-4xCO2']
source_id = ['CIESM', 'CMCC-CM2-SR5', 'CMCC-ESM2', 'CanESM5', 'EC-Earth3-Veg', 'GISS-E2-1-G', 'GISS-E2-2-G', 'MIROC6', 'MPI-ESM1-2-HR', 'MPI-ESM1-2-LR', 'MRI-ESM2-0']
source_id = sorted(source_id) #making sure the models are listed in alphabetical order

In [4]:
# Sorting the data alphabetically, making sure we've got what we need

cat1 = col.search(experiment_id = experiment_id, table_id = 'Omon', variable_id = 'hfbasin', activity_id = 'CMIP', member_id = 'r1i1p1f1', source_id = source_id) 

data = cat1.df

df_sorted = data.sort_values(by=[data.columns[2], data.columns[3]], ascending=True) #HERE! The alphabetical sorting happened here and the source_id was NOT alphabetically sorted!!!!!

# Reset index
df_sorted.reset_index(drop=True, inplace=True)

HTML(df_sorted.to_html())

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
0,CMIP,THU,CIESM,abrupt-4xCO2,r1i1p1f1,Omon,hfbasin,gn,gs://cmip6/CMIP6/CMIP/THU/CIESM/abrupt-4xCO2/r1i1p1f1/Omon/hfbasin/gn/v20200220/,NaN,20200220
1,CMIP,THU,CIESM,piControl,r1i1p1f1,Omon,hfbasin,gn,gs://cmip6/CMIP6/CMIP/THU/CIESM/piControl/r1i1p1f1/Omon/hfbasin/gn/v20200220/,NaN,20200220
2,CMIP,CMCC,CMCC-CM2-SR5,abrupt-4xCO2,r1i1p1f1,Omon,hfbasin,gn,gs://cmip6/CMIP6/CMIP/CMCC/CMCC-CM2-SR5/abrupt-4xCO2/r1i1p1f1/Omon/hfbasin/gn/v20200616/,NaN,20200616
3,CMIP,CMCC,CMCC-CM2-SR5,piControl,r1i1p1f1,Omon,hfbasin,gn,gs://cmip6/CMIP6/CMIP/CMCC/CMCC-CM2-SR5/piControl/r1i1p1f1/Omon/hfbasin/gn/v20200616/,NaN,20200616
4,CMIP,CMCC,CMCC-ESM2,abrupt-4xCO2,r1i1p1f1,Omon,hfbasin,gn,gs://cmip6/CMIP6/CMIP/CMCC/CMCC-ESM2/abrupt-4xCO2/r1i1p1f1/Omon/hfbasin/gn/v20210114/,NaN,20210114
5,CMIP,CMCC,CMCC-ESM2,piControl,r1i1p1f1,Omon,hfbasin,gn,gs://cmip6/CMIP6/CMIP/CMCC/CMCC-ESM2/piControl/r1i1p1f1/Omon/hfbasin/gn/v20210304/,NaN,20210304
6,CMIP,CCCma,CanESM5,abrupt-4xCO2,r1i1p1f1,Omon,hfbasin,gn,gs://cmip6/CMIP6/CMIP/CCCma/CanESM5/abrupt-4xCO2/r1i1p1f1/Omon/hfbasin/gn/v20190429/,NaN,20190429
7,CMIP,CCCma,CanESM5,piControl,r1i1p1f1,Omon,hfbasin,gn,gs://cmip6/CMIP6/CMIP/CCCma/CanESM5/piControl/r1i1p1f1/Omon/hfbasin/gn/v20190429/,NaN,20190429
8,CMIP,EC-Earth-Consortium,EC-Earth3-Veg,abrupt-4xCO2,r1i1p1f1,Omon,hfbasin,gn,gs://cmip6/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg/abrupt-4xCO2/r1i1p1f1/Omon/hfbasin/gn/v20200919/,NaN,20200919
9,CMIP,EC-Earth-Consortium,EC-Earth3-Veg,piControl,r1i1p1f1,Omon,hfbasin,gn,gs://cmip6/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg/piControl/r1i1p1f1/Omon/hfbasin/gn/v20200919/,NaN,20200919


In [5]:
# Loading the data, putting everything into a list

dset_dict = cat1.to_dataset_dict(zarr_kwargs={'consolidated': True})
models_experiments_list = list(dset_dict.keys())


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


In [6]:
# Model names
models_experiments = []

for dataset in models_experiments_list:
    names = dataset.split('.')    
    dataset_name = names[2] + '.' + names[3]        
    models_experiments.append(dataset_name)

In [7]:
# Function to extract model name
def extract_model_name(string_name):
    return string_name.split('.')[2]

In [8]:
# Separating the datasets in piControl, x4CO2 lists and sorting them in alphabetical order.

ds_piControl = []
ds_x4CO2 = []

# Extracting the names of models from the dataset names

piControl_names = [name for name in models_experiments_list if 'piControl' in name]
abrupt4xCO2_names = [name for name in models_experiments_list if 'abrupt-4xCO2' in name]

# Now, we can sort the lists alphabetically by the MODEL name and not just the dataframe name

piControl_names.sort(key=extract_model_name)
abrupt4xCO2_names.sort(key=extract_model_name)

# Dropping everything in the correct list

for i, j in zip(piControl_names, abrupt4xCO2_names):
    
    ds_piControl.append(dset_dict[i])
    ds_x4CO2.append(dset_dict[j])

In [68]:
# Choosing the time period

# years 0-150:

start_y = 0
end_y = 1800

period = 1789

start_number = 0 # for plot labeling
end_number = 150

# years 50-150:

#start_y = 600
#end_y = 1800

#period = 1189 #(1200-12+1)

#start_number = 50 # for plot labeling
#end_number = 150

# years 100-150

#start_y = 1200
#end_y = 1800

#period = 589 #(600-12+1)

#start_number = 100 # for plot labeling
#end_number = 150

In [10]:
# Different models have different names for each basin. Here, we rename everything so they all have the same name.
name_mapping = {
    "Global": "global_ocean",
    "global_ocean": "global_ocean",
    "global": "global_ocean",
    "g": "global_ocean",  

    "Atlantic": "atlantic_arctic_ocean",    
    "atlantic_ocean": "atlantic_arctic_ocean",
    "atlantic_arctic_ocean": "atlantic_arctic_ocean",
    "atlantic": "atlantic_arctic_ocean",
    "a": "atlantic_arctic_ocean",    

    "Indo-Pacific": "indian_pacific_ocean",
    "indian_pacific_ocean": "indian_pacific_ocean",
    "indian-pacific": "indian_pacific_ocean",
    "i": "indian_pacific_ocean"       
}

# Function to process data and extract the global_ocean basin 
def basin_separation_I(data, model_name):
    global_data = []
    atlantic_data = []
    indian_pacific_data = []
    
    if model_name == 'IPSL-CM6A-LR':
        
        data = data.rename({'3basin': 'basin'})
        basin_labels = ['Global', 'Atlantic', 'indian_pacific_ocean']
        basin_indexes = range(len(basin_labels))      

    elif model_name == 'IPSL-CM6A-MR1':
        
        basin_labels = ['Global', 'Atlantic', 'indian_pacific_ocean']
        basin_indexes = range(len(basin_labels))  

    elif model_name == 'NorESM2-LM':
        basin_labels = ['atlantic_arctic_ocean', 'atlantic_arctic_extended_ocean', 'indian_pacific_ocean', 'global_ocean']
        basin_indexes = range(len(basin_labels))   

    #For the cases where the basin names are encoded and need to be extracted        
    elif 'sector' in data.coords and 'basin' in data.dims:
        
        basin_labels = data['sector'].values
        #basin_labels = [label.decode('utf-8').strip() for label in basin_labels]
        basin_labels = [label.decode('utf-8').strip() if isinstance(label, bytes) else str(label).strip() for label in basin_labels]
        basin_indexes = data['basin'].values
        
    elif 'basin' in data.dims and 'requested' in data['basin'].attrs:
        
        basin_labels = [k.split('=')[0].strip() for k in requested_attr.split(', ')]
        basin_indexes = data['basin'].values
        requested_attr = data['basin'].attrs['requested']        
        
    else:
        print(f"{model_name} : No valid basin names found.")
        return []

    # Create a mapping of indexes to standardized names
    basin_mapping = {}
    
    for idx, label in zip(basin_indexes, basin_labels):
        
        cleaned_label = label.strip()  # strip removes all potential white spaces #.lower() #turns all letters to to lower-case
        standardized_name = name_mapping.get(cleaned_label, f"unknown_{label}")
        print(f"Original: '{label}', Cleaned: '{cleaned_label}', Mapped: '{standardized_name}'")  # Debugging
        basin_mapping[idx] = standardized_name

    print("Basin Mapping:", basin_mapping)

    # Extract data for the global_ocean basin
    for idx, name in basin_mapping.items():

        basin_data = data.isel(basin=idx)  # Extract data for the basin
        
        if name == "global_ocean":
            global_data.append(basin_data)

        elif name == "atlantic_arctic_ocean":
            atlantic_data.append(basin_data)

        elif name == "indian_pacific_ocean":
            indian_pacific_data.append(basin_data)
            
    return global_data, atlantic_data, indian_pacific_data

# Merged script

In [12]:
# Lists to loop over. 

models_pangeo = source_id
models_esgf = ['CanESM5-1', 'EC-Earth3-AerChem', 'EC-Earth3-CC', 'EC-Earth3-Veg-LR', 'IPSL-CM6A-LR', 'IPSL-CM6A-MR1', 'NorESM2-LM', 'NorESM2-MM']
models_esgf = sorted(models_esgf)
#Same as before, we only work with the models that also include hfbasin.
#Original list with the models that contain all 9 heat fluxes:['CanESM5-1', 'CAS-ESM2-0', 'CIESM', 'CMCC-ESM2', 'E3SM-2-0', 'EC-Earth3-AerChem', 'EC-Earth3-CC', 'EC-Earth3-Veg', 'EC-Earth3-Veg-LR', 'GISS-E2-2-H', 'IPSL-CM6A-LR', 'IPSL-CM6A-MR1', 'MPI-ESM-1-2-HAM', 'MRI-ESM2-0', 'NESM3', 'NorESM2-LM']

models_all = sorted(models_pangeo + models_esgf) #alphabetical order

experiments_pangeo = [ds_piControl, ds_x4CO2]
experiments_esgf = ['piControl', 'x4CO2']

In [70]:
models_all

['CIESM',
 'CMCC-CM2-SR5',
 'CMCC-ESM2',
 'CanESM5',
 'CanESM5-1',
 'EC-Earth3-AerChem',
 'EC-Earth3-CC',
 'EC-Earth3-Veg',
 'EC-Earth3-Veg-LR',
 'GISS-E2-1-G',
 'GISS-E2-2-G',
 'IPSL-CM6A-LR',
 'IPSL-CM6A-MR1',
 'MIROC6',
 'MPI-ESM1-2-HR',
 'MPI-ESM1-2-LR',
 'MRI-ESM2-0',
 'NorESM2-LM',
 'NorESM2-MM']

In [100]:
# Do you want time average or not? True: Yes plese, False: Not thank you

time_average = False

num_y = 150 # the number of years I'm working with when I'm not using time average

# Initialize dictionaries to hold the component arrays for each model
lat_basin_model = []

hfbasins_gl_model = []
hfbasins_atl_model = []
hfbasins_ip_model = []

hfbasins_gl_model_t = []
hfbasins_atl_model_t = []
hfbasins_ip_model_t = []

for model in models_all:
        
    if model in models_pangeo: #already alphabetically ordered, so no problem in the big alphabetical list
        
        model_idx = models_pangeo.index(model)
        
        hfbasins_gl_exp = []
        hfbasins_atl_exp = []
        hfbasins_ip_exp = []
        
        lat_basin_exp = []
        
        for experiment in experiments_pangeo:
            # Initialize dictionaries to hold the component arrays for each model

            model_exp = experiment[model_idx]
            
            string_name = model_exp.attrs.get('intake_esm_dataset_key')
            model_name = extract_model_name(string_name)
            print(model_name, ' is a pangeo model')
        
            if time_average == True:
                model_50 = model_exp.isel(time=slice(start_y, end_y)).mean(dim='time') #time average
            else:
                model_50 = model_exp.isel(time=slice(start_y, end_y))
    
            lat =  model_50['lat'].values
            dask.compute()
                
            component_values = model_50['hfbasin'][0, 0, :, :]
    
            global_data, atlantic_data, indian_pacific_data = basin_separation_I(component_values, model_name)
                
            for global_basin_data, atlantic_basin_data, indian_pacific_basin_data in zip(global_data, atlantic_data, indian_pacific_data):

                if time_average == False: #turning monthly intoyearly data

                    global_basin_data = global_basin_data.values * 10**-15
                    atlantic_basin_data = atlantic_basin_data.values * 10**-15
                    indian_pacific_basin_data = indian_pacific_basin_data.values * 10**-15

                    global_basin_data_reshaped = global_basin_data.reshape(num_y, 12, len(lat))
                    global_basin_data = np.mean(global_basin_data_reshaped, axis=1)
        
                    atlantic_basin_data_reshaped = atlantic_basin_data.reshape(num_y, 12, len(lat))
                    atlantic_basin_data = np.mean(atlantic_basin_data_reshaped, axis=1)
        
                    indian_pacific_basin_data_reshaped = indian_pacific_basin_data.reshape(num_y, 12, len(lat))
                    indian_pacific_basin_data = np.mean(indian_pacific_basin_data_reshaped, axis=1)
                    
                hfbasins_gl_exp.append(global_basin_data) 
                lat_basin_exp.append(lat)
    
                hfbasins_atl_exp.append(atlantic_basin_data) 
    
                hfbasins_ip_exp.append(indian_pacific_basin_data)  
                
        lat_basin_model.append(lat_basin_exp) 
        
        if time_average ==True:
            hfbasins_gl_model.append(hfbasins_gl_exp) 
            hfbasins_atl_model.append(hfbasins_atl_exp)   
            hfbasins_ip_model.append(hfbasins_ip_exp) 
        
        else:        
            hfbasins_gl_model_t.append(hfbasins_gl_exp) 
            hfbasins_atl_model_t.append(hfbasins_atl_exp) 
            hfbasins_ip_model_t.append(hfbasins_ip_exp) 

    elif model in models_esgf:

        hfbasins_gl_exp = []        
        hfbasins_atl_exp = []
        hfbasins_ip_exp = []
        
        lat_basin_exp = []
        
        for experiment in experiments_esgf:
            print(model, ' is an esgf model')

            folder_path = f'/mn/vann/chrikap/CMIP6/{model}/{experiment}/OCEAN/hfbasin'
            file_names = sorted(os.listdir(folder_path))
    
            # Using dask to read datasets
            datasets = [xr.open_dataset(os.path.join(folder_path, file_name), chunks={'time': 10}) for file_name in file_names if file_name.endswith('.nc')]    
    
            if not datasets:
                print(f"Skipping {model} due to missing datasets")
                continue  # This model gets skipped
    
            if len(datasets) == 1:
                ds = datasets[0]
                model_50 = ds.isel(time=slice(start_y, end_y))
            else:
                # Merge all the datasets in the list along the same dimension
                merged_datasets = xr.concat(datasets, dim='time')
                model_50 = merged_datasets.isel(time=slice(start_y, end_y))

            if time_average == True:
                model_50 = model_50.mean(dim='time')
    
            lat =  model_50['nav_lat'][:, 0].values if model in ['IPSL-CM6A-LR', 'IPSL-CM6A-MR1'] else model_50['lat'].values
            dask.compute()
                
            component_values = model_50['hfbasin']
            
            global_data, atlantic_data, indian_pacific_data = basin_separation_I(component_values, model)
                
            for global_basin_data, atlantic_basin_data, indian_pacific_basin_data in zip(global_data, atlantic_data, indian_pacific_data):
    
                if global_basin_data.shape == (332, 1):  #just for the ipsl model- i have to fix how t make it more general later
                    global_basin_data = global_basin_data[:, 0]
                    atlantic_basin_data = atlantic_basin_data[:, 0]
                    indian_pacific_basin_data = indian_pacific_basin_data[:, 0]
    
                # turning W->PW has to happen now because interpolation will be difficult with such high values later

                if time_average == False: #turning monthly intoyearly data

                    global_basin_data = global_basin_data.values * 10**-15
                    atlantic_basin_data = atlantic_basin_data.values * 10**-15
                    indian_pacific_basin_data = indian_pacific_basin_data.values * 10**-15

                    global_basin_data_reshaped = global_basin_data.reshape(num_y, 12, len(lat))
                    global_basin_data = np.mean(global_basin_data_reshaped, axis=1)
        
                    atlantic_basin_data_reshaped = atlantic_basin_data.reshape(num_y, 12, len(lat))
                    atlantic_basin_data = np.mean(atlantic_basin_data_reshaped, axis=1)
        
                    indian_pacific_basin_data_reshaped = indian_pacific_basin_data.reshape(num_y, 12, len(lat))
                    indian_pacific_basin_data = np.mean(indian_pacific_basin_data_reshaped, axis=1)
                    
                hfbasins_gl_exp.append(global_basin_data) 
                lat_basin_exp.append(lat)
    
                hfbasins_atl_exp.append(atlantic_basin_data) 
    
                hfbasins_ip_exp.append(indian_pacific_basin_data)  
       
        lat_basin_model.append(lat_basin_exp)

        if time_average == True:
            hfbasins_gl_model.append(hfbasins_gl_exp)
            hfbasins_atl_model.append(hfbasins_atl_exp)        
            hfbasins_ip_model.append(hfbasins_ip_exp)
    
        else:

            hfbasins_gl_model_t.append(hfbasins_gl_exp)    
            hfbasins_atl_model_t.append(hfbasins_atl_exp)    
            hfbasins_ip_model_t.append(hfbasins_ip_exp)

CIESM  is a pangeo model
Original: 'a', Cleaned: 'a', Mapped: 'atlantic_arctic_ocean'
Original: 'i', Cleaned: 'i', Mapped: 'indian_pacific_ocean'
Original: 'g', Cleaned: 'g', Mapped: 'global_ocean'
Basin Mapping: {0: 'atlantic_arctic_ocean', 1: 'indian_pacific_ocean', 2: 'global_ocean'}
CIESM  is a pangeo model
Original: 'a', Cleaned: 'a', Mapped: 'atlantic_arctic_ocean'
Original: 'i', Cleaned: 'i', Mapped: 'indian_pacific_ocean'
Original: 'g', Cleaned: 'g', Mapped: 'global_ocean'
Basin Mapping: {0: 'atlantic_arctic_ocean', 1: 'indian_pacific_ocean', 2: 'global_ocean'}
CMCC-CM2-SR5  is a pangeo model
Original: 'a', Cleaned: 'a', Mapped: 'atlantic_arctic_ocean'
Original: 'i', Cleaned: 'i', Mapped: 'indian_pacific_ocean'
Original: 'g', Cleaned: 'g', Mapped: 'global_ocean'
Basin Mapping: {0: 'atlantic_arctic_ocean', 1: 'indian_pacific_ocean', 2: 'global_ocean'}
CMCC-CM2-SR5  is a pangeo model
Original: 'a', Cleaned: 'a', Mapped: 'atlantic_arctic_ocean'
Original: 'i', Cleaned: 'i', Mapped:

In [16]:
# Regridding the curves along latitude

target_lat = np.arange(-90, 92, 2)  # Define the target latitude grid

In [53]:
# Time averaged values
hfbasins_gl_int_model = []
hfbasins_atl_int_model = []
hfbasins_ip_int_model = []

for oht_gl_exp, oht_atl_exp, oht_ip_exp, lat_exp in zip(hfbasins_gl_model, hfbasins_atl_model, hfbasins_ip_model, lat_basin_model):

    hfbasin_gl_int_exp = []
    hfbasin_atl_int_exp = []
    hfbasin_ip_int_exp = []
    
    for oht_gl, oht_atl, oht_ip, lat_v in zip(oht_gl_exp, oht_atl_exp, oht_ip_exp, lat_exp):

        old_lat = np.array(lat_v)

        gl_interp_func = interp1d(old_lat, oht_gl, kind='linear', bounds_error=False, fill_value='extrapolate')
        atl_interp_func = interp1d(old_lat, oht_atl, kind='linear', bounds_error=False, fill_value='extrapolate')
        ip_interp_func = interp1d(old_lat, oht_ip, kind='linear', bounds_error=False, fill_value='extrapolate')

        gl_interp = gl_interp_func(target_lat)
        atl_interp = atl_interp_func(target_lat)
        ip_interp = ip_interp_func(target_lat)

        hfbasin_gl_int_exp.append(gl_interp)
        hfbasin_atl_int_exp.append(atl_interp)
        hfbasin_ip_int_exp.append(ip_interp)        

    hfbasins_gl_int_model.append(hfbasin_gl_int_exp)
    hfbasins_atl_int_model.append(hfbasin_atl_int_exp)
    hfbasins_ip_int_model.append(hfbasin_ip_int_exp)

hfbasins_gl_int_model = np.array(hfbasins_gl_int_model)
hfbasins_atl_int_model = np.array(hfbasins_atl_int_model)
hfbasins_ip_int_model = np.array(hfbasins_ip_int_model)

In [54]:
# There's a drop in the ip oht at 0-20N. A lot of land there, so that's probably the reason why. 

(19, 2, 91)

In [67]:
# Storing everything because re-running the script takes a lot of time

# Time averages
os.chdir('/home/chrikap/Desktop/atmospheric heat transport/heat transport datasets/Annual_Variations/CMIP6/Paper_Scripts/heat_transports')

np.save('hfbasins_gl_int_model', hfbasins_gl_int_model)
np.save('hfbasins_atl_int_model', hfbasins_atl_int_model)
np.save('hfbasins_ip_int_model', hfbasins_ip_int_model)

In [101]:
# No time average

# Time averaged values
oht_gl_int_model_t = []
oht_atl_int_model_t = []
oht_ip_int_model_t = []

for oht_gl_exp, oht_atl_exp, oht_ip_exp, lat_exp in zip(hfbasins_gl_model_t, hfbasins_atl_model_t, hfbasins_ip_model_t, lat_basin_model):

    oht_gl_int_exp = []
    oht_atl_int_exp = []
    oht_ip_int_exp = []
    
    for oht_gl, oht_atl, oht_ip, lat_v in zip(oht_gl_exp, oht_atl_exp, oht_ip_exp, lat_exp):
        
        oht_gl_int_time = []
        oht_atl_int_time = []
        oht_ip_int_time = []

        old_lat = np.array(lat_v)

        for oht_gl_time, oht_atl_time, oht_ip_time in zip(oht_gl, oht_atl, oht_ip):

            oht_gl_interp_func = interp1d(old_lat, oht_gl_time, kind='linear', bounds_error=False, fill_value='extrapolate')
            oht_atl_interp_func = interp1d(old_lat, oht_atl_time, kind='linear', bounds_error=False, fill_value='extrapolate')
            oht_ip_interp_func = interp1d(old_lat, oht_ip_time, kind='linear', bounds_error=False, fill_value='extrapolate')
    
            oht_gl_interp = oht_gl_interp_func(target_lat)
            oht_atl_interp = oht_atl_interp_func(target_lat)
            oht_ip_interp = oht_ip_interp_func(target_lat)
    
            oht_gl_int_time.append(oht_gl_interp)
            oht_atl_int_time.append(oht_atl_interp)
            oht_ip_int_time.append(oht_ip_interp)
            
        oht_gl_int_exp.append(oht_gl_int_time)
        oht_atl_int_exp.append(oht_atl_int_time)            
        oht_ip_int_exp.append(oht_ip_int_time)
        
    oht_gl_int_model_t.append(oht_gl_int_exp)
    oht_atl_int_model_t.append(oht_atl_int_exp)
    oht_ip_int_model_t.append(oht_ip_int_exp)

oht_gl_int_model_t = np.array(oht_gl_int_model_t)
oht_atl_int_model_t = np.array(oht_atl_int_model_t)
oht_ip_int_model_t = np.array(oht_ip_int_model_t)

In [108]:
# Storing everything because re-running the script takes a lot of time

# Time averages
os.chdir('/home/chrikap/Desktop/atmospheric heat transport/heat transport datasets/Annual_Variations/CMIP6/Paper_Scripts/heat_transports')

np.save('oht_gl_int_model_t', oht_gl_int_model_t)
np.save('oht_atl_int_model_t', oht_atl_int_model_t)
np.save('oht_ip_int_model_t', oht_ip_int_model_t)